In [2]:
from helper import DataPreprocessing
from ipywidgets import interact, IntSlider, FloatSlider
import pywt 
import matplotlib.pyplot as plt
import numpy as np
import librosa
#%matplotlib nbagg

In [2]:
def peak_frequency (spec,freq):
    peak_freqs = np.zeros(spec.shape[1])
    for i in range(spec.shape[1]):
        idx = np.argmax(spec[:, i])
        peak_freqs[i] = freq[idx] 
    return peak_freqs

In [3]:
def load_train_full_cwt (wavelet_name,scales,kind):

    if kind == "train":
        bearing_number =  [11,12,21,22,31,32]
    else:
        bearing_number =  [13,14,15,16,17,23,24,25,26,27,33]
    @interact(bearing = bearing_number)
    def load_data_train(bearing):
        
        #load method
        mt = "Continuous Wavelet Transform"
        method = "CWT"

        #load data
        data = DataPreprocessing(kind)
    
        N = data.n_samples(kind,bearing)
        eol = data.eol(bearing)
        time_stamp = np.linspace(0,eol,N)
        stand = data.stand(bearing)
        hor_acc = stand[0]
        ver_acc = stand[1]
        w_size = data.window_size_gaf
        sr = data.sr
        si = 1/sr

        @interact(label = IntSlider(min=0,max=eol,step=10))
        def visualize_CWT(label):
          
            #scale data with label and step
            s_step = round(N - label * sr / 100 - w_size)
            time_stamp_w = time_stamp[s_step:s_step+w_size]
            time_stamp_w_real = np.linspace((eol-label)+0,(eol-label)+0.1,w_size)
            hor_acc_w = hor_acc[s_step:s_step+w_size]
            ver_acc_w = ver_acc[s_step:s_step+w_size]
            t_step = time_stamp_w_real.min()

            #visualize
            fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
            fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

            #axes 1:
            ax1.plot(time_stamp,hor_acc)
            ax1.plot(time_stamp_w,hor_acc_w)
            ax1.legend(["horizontal","window"],loc="upper left")
            ax1.set_ylabel("Normalized value")
            ax1.set_xlabel("Timestamp")
            ax1.set_title("Horizontal Acceleromete, time step {}".format(int(t_step)))

            #axes 2:
            ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
            ax2.legend(["horizontal window"],loc = "upper left")
            ax2.set_ylabel("Normalized value")
            ax2.set_xlabel("Timestamp window")
            ax2.set_title("Horizontal Acceleromete Windowm, label {}".format(label))
            ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 3:
            cwt_hor,freq = pywt.cwt(data=hor_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_hor = np.abs(cwt_hor)
            spec_hor = librosa.power_to_db(spec_hor)
            #image_cwt_hor = ax3.imshow(spec_hor, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_hor = ax3.pcolormesh(time_stamp_w_real , freq, spec_hor)
            plt.colorbar(image_cwt_hor,ax=ax3)
            ax3.set_title("{}, size {}".format(method,cwt_hor.shape))
            #ax3.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax3.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax3.set_ylabel("Frequency")
            ax3.set_xlabel("Time Doamin")
            
            #axes 4:
            peak_hor = peak_frequency(spec_hor,freq)
            ax4.plot(np.linspace((eol-label),(eol-label)+0.1,spec_hor.shape[1]),peak_hor)
            ax4.set_xlabel("Time Domain")
            ax4.set_ylabel("Peak Frequency")
            ax4.set_title("Peak Frequency")            

            #axes 5:
            ax5.plot(time_stamp,ver_acc)
            ax5.plot(time_stamp_w,ver_acc_w)
            ax5.legend(["vertikal","window"],loc="upper left")
            ax5.set_ylabel("Normalized value")
            ax5.set_xlabel("Timestamp")
            ax5.set_title("Vertikal Acceleromete,time step {}".format(int(t_step)))
            
            #axes 6:
            ax6.plot(time_stamp_w_real,ver_acc_w,"darkorange")
            ax6.legend(["vertikal window"],loc = "upper left")
            ax6.set_ylabel("Normalized value")
            ax6.set_xlabel("Timestamp window")
            ax6.set_title("Horizontal Acceleromete Window label {}".format(label))
            ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 7:
            cwt_ver,freq =pywt.cwt(data=ver_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_ver = np.abs(cwt_ver)
            spec_ver = librosa.power_to_db(spec_ver)
            #image_cwt_ver = ax7.imshow(spec_ver, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_ver = ax7.pcolormesh(time_stamp_w_real , freq, spec_ver)
            plt.colorbar(image_cwt_ver,ax=ax7)
            ax7.set_title("{}, size {}".format(method,cwt_ver.shape))
            #ax7.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax7.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax7.set_ylabel("Frequency")
            ax7.set_xlabel("Time Doamin")

            #axes 8:
            peak_ver = peak_frequency(spec_ver,freq)
            ax8.set_xlabel("Time Domain")
            ax8.set_ylabel("Peak Frequency")
            ax8.set_title("Peak Frequency")
            ax8.plot(np.linspace((eol-label),(eol-label)+0.1,spec_ver.shape[1]),peak_ver) 

            print("Done")
                

In [4]:
def load_test_cwt (wavelet_name,scales,kind):
    @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
    def load_data_test(bearing):
        
        #load method
        mt = "Continuous Wavelet Transform"
        method = "CWT"  

        #load data
        data = DataPreprocessing(kind)

        N = data.n_samples(kind,bearing)
        last_cycle = data.last_cycle(bearing)
        eol = data.eol(bearing)
        rul = data.rul(bearing)
        time_stamp = np.linspace(0,last_cycle,N)
        stand = data.stand(bearing)
        w_size = data.window_size 
        hor_acc = stand[0]
        ver_acc = stand[1]
        sr = data.sr
        si = 1/sr

        @interact(label = IntSlider(min=rul,max=rul+last_cycle,step=10))
        def visualize_CWT (label):

            #scale data with label and step
            s_step = round(N - (label-rul) * sr / 100 - w_size)
            time_stamp_w = time_stamp[s_step:s_step+w_size]
            time_stamp_w_real = np.linspace((last_cycle-label+rul)+0,(last_cycle-label+rul)+0.1,w_size)
            hor_acc_w = hor_acc[s_step:s_step+w_size]
            ver_acc_w = ver_acc[s_step:s_step+w_size]
            t_step = time_stamp_w_real.min()

            #visualize
            fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
            fig.suptitle("{} data, bearing {}, EOL {} s, RUL {} s, {} samples\nMethod {}".format(kind,bearing,eol,rul,N,mt))

            #axes 1:
            ax1.plot(time_stamp,hor_acc)
            ax1.plot(time_stamp_w,hor_acc_w)
            ax1.legend(["horizontal","window"],loc="upper left")
            ax1.set_ylabel("Normalized value")
            ax1.set_xlabel("Timestamp")
            ax1.set_title("Horizontal Acceleromete, step {}".format(int(t_step)))

            #axes 2:
            ax2.plot(time_stamp_w_real,hor_acc_w,c="darkorange")
            ax2.legend(["horizontal window"],loc = "upper left")
            ax2.set_ylabel("Normalized value")
            ax2.set_xlabel("Timestamp window")
            ax2.set_title("Horizontal Acceleromete Window, label {}".format(label))
            ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 3:
            cwt_hor,freq = pywt.cwt(data=hor_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_hor = np.abs(cwt_hor)
            spec_hor = librosa.power_to_db(spec_hor)
            #image_cwt_hor = ax3.imshow(spec_hor, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_hor = ax3.pcolormesh(time_stamp_w_real , freq, spec_hor)
            plt.colorbar(image_cwt_hor,ax=ax3)
            ax3.set_title("{}, size {}".format(method,cwt_hor.shape))
            #ax3.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax3.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax3.set_ylabel("Frequency")
            ax3.set_xlabel("Time Doamin")
            
            #axes 4:
            peak_hor = peak_frequency(spec_hor,freq)
            ax4.plot(np.linspace((last_cycle-label+rul),(last_cycle-label+rul)+0.1,spec_hor.shape[1]),peak_hor)
            ax4.set_xlabel("Time Domain")
            ax4.set_ylabel("Peak Frequency")
            ax4.set_title("Peak Frequency")

            #axes 5:
            ax5.plot(time_stamp,ver_acc)
            ax5.plot(time_stamp_w,ver_acc_w)
            ax5.legend(["vertikal","window"],loc="upper left")
            ax5.set_ylabel("Normalized value")
            ax5.set_xlabel("Timestamp")
            ax5.set_title("Vertikal Acceleromete, step {}".format(int(t_step)))
            
            #axes 6:
            ax6.plot(time_stamp_w_real,ver_acc_w,c="darkorange")
            ax6.legend(["vertikal window"],loc = "upper left")
            ax6.set_ylabel("Normalized value")
            ax6.set_xlabel("Timestamp")
            ax6.set_title("Horizontal Acceleromete Window, label {}".format(label))
            ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 7:
            cwt_ver,freq =pywt.cwt(data=ver_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_ver = np.abs(cwt_ver)
            spec_ver = librosa.power_to_db(spec_ver)
            #image_cwt_ver = ax7.imshow(spec_ver, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_ver = ax7.pcolormesh(time_stamp_w_real , freq, spec_ver)
            plt.colorbar(image_cwt_ver,ax=ax7)
            ax7.set_title("{}, size {}".format(method,cwt_ver.shape))
            #ax7.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax7.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax7.set_ylabel("Frequency Domain")
            ax7.set_xlabel("Time Doamin")

            #axes 8:
            peak_ver = peak_frequency(spec_ver,freq)
            ax8.set_xlabel("Time Domain")
            ax8.set_ylabel("Peak Frequency")
            ax8.set_title("Peak Frequency")
            ax8.plot(np.linspace((last_cycle-label+rul),(last_cycle-label+rul)+0.1,spec_ver.shape[1]),peak_ver)

            print("Done")

In [5]:

def load_full_cwt (wavelet_name,scales,kind):
    @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
    def load_data_train(bearing):
        
        #load method
        mt = "Continuous Wavelet Transform"
        method = "CWT"

        #load data
        data = DataPreprocessing(kind)
    
        N = data.n_samples(kind,bearing)
        eol = data.eol(bearing)
        time_stamp = np.linspace(0,eol,N)
        stand = data.stand(bearing)
        hor_acc = stand[0]
        ver_acc = stand[1]
        w_size = data.window_size_gaf
        sr = data.sr
        si = 1/sr

        @interact(label = IntSlider(min=0,max=eol,step=10))
        def visualize_CWT(label):
          
            #scale data with label and step
            s_step = round(N - label * sr / 100 - w_size)
            time_stamp_w = time_stamp[s_step:s_step+w_size]
            time_stamp_w_real = np.linspace((eol-label)+0,(eol-label)+0.1,w_size)
            hor_acc_w = hor_acc[s_step:s_step+w_size]
            ver_acc_w = ver_acc[s_step:s_step+w_size]
            t_step = time_stamp_w_real.min()

            #visualize
            fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
            fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

            #axes 1:
            ax1.plot(time_stamp,hor_acc)
            ax1.plot(time_stamp_w,hor_acc_w)
            ax1.legend(["horizontal","window"],loc="upper left")
            ax1.set_ylabel("Normalized value")
            ax1.set_xlabel("Timestamp")
            ax1.set_title("Horizontal Acceleromete, time step {}".format(int(t_step)))

            #axes 2:
            ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
            ax2.legend(["horizontal window"],loc = "upper left")
            ax2.set_ylabel("Normalized value")
            ax2.set_xlabel("Timestamp window")
            ax2.set_title("Horizontal Acceleromete Windowm, label {}".format(label))
            ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 3:
            cwt_hor,freq = pywt.cwt(data=hor_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_hor = np.abs(cwt_hor)
            spec_hor = librosa.power_to_db(spec_hor)
            #image_cwt_hor = ax3.imshow(spec_hor, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_hor = ax3.pcolormesh(time_stamp_w_real , freq, spec_hor)
            plt.colorbar(image_cwt_hor,ax=ax3)
            ax3.set_title("{}, size {}".format(method,cwt_hor.shape))
            #ax3.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax3.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax3.set_ylabel("Frequency")
            ax3.set_xlabel("Time Doamin")
            
            #axes 4:
            peak_hor = peak_frequency(spec_hor,freq)
            ax4.plot(np.linspace((eol-label),(eol-label)+0.1,spec_hor.shape[1]),peak_hor)
            ax4.set_xlabel("Time Domain")
            ax4.set_ylabel("Peak Frequency")
            ax4.set_title("Peak Frequency")            

            #axes 5:
            ax5.plot(time_stamp,ver_acc)
            ax5.plot(time_stamp_w,ver_acc_w)
            ax5.legend(["vertikal","window"],loc="upper left")
            ax5.set_ylabel("Normalized value")
            ax5.set_xlabel("Timestamp")
            ax5.set_title("Vertikal Acceleromete,time step {}".format(int(t_step)))
            
            #axes 6:
            ax6.plot(time_stamp_w_real,ver_acc_w,"darkorange")
            ax6.legend(["vertikal window"],loc = "upper left")
            ax6.set_ylabel("Normalized value")
            ax6.set_xlabel("Timestamp window")
            ax6.set_title("Horizontal Acceleromete Window label {}".format(label))
            ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

            #axes 7:
            cwt_ver,freq =pywt.cwt(data=ver_acc_w,scales=scales,wavelet=wavelet_name,sampling_period=si)
            spec_ver = np.abs(cwt_ver)
            spec_ver = librosa.power_to_db(spec_ver)
            #image_cwt_ver = ax7.imshow(spec_ver, extent=[t_step, t_step+0.1, scales.max(), scales.min()], interpolation="none", aspect="auto")
            image_cwt_ver = ax7.pcolormesh(time_stamp_w_real , freq, spec_ver)
            plt.colorbar(image_cwt_ver,ax=ax7)
            ax7.set_title("{}, size {}".format(method,cwt_ver.shape))
            #ax7.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
            #ax7.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
            ax7.set_ylabel("Frequency")
            ax7.set_xlabel("Time Doamin")

            #axes 8:
            peak_ver = peak_frequency(spec_ver,freq)
            ax8.set_xlabel("Time Domain")
            ax8.set_ylabel("Peak Frequency")
            ax8.set_title("Peak Frequency")
            ax8.plot(np.linspace((eol-label),(eol-label)+0.1,spec_ver.shape[1]),peak_ver) 

            print("Done")

In [6]:
@interact(kind = ["train","full","test"], wavelet = ["cmor","cgau","morl","mexh","gaus"], scale_min = IntSlider(min = 1, max=20, step=1,value=2), scale_max = IntSlider(min = 2, max=200, step=1,value=20), n_scale = IntSlider(min=10,max=800,value=250))
def choose_kind_data(kind, wavelet,scale_min, scale_max, n_scale):

    #scale
    scales = np.linspace(scale_min,scale_max,n_scale)
    #scales = np.logspace(np.log10(10), np.log10(scale_max),n_scale)
    if wavelet == "cmor": 
        @interact(B = FloatSlider(min=0.1,max=5,step=0.1,value=1.5), C = FloatSlider(min=0.1,max=5,step=0.1,value=1))
        def wavelet (B,C):
            #load wavelet name
            wavelet_name = "cmor{}-{}".format(B,C)

            #load kind of data
            if kind == "train" or kind == "full":
                load_train_full_cwt(wavelet_name,scales,kind)
            elif kind == "test":
                load_test_cwt (wavelet_name,scales,kind)

    elif wavelet in ["morl", "mexh"]:
        #load wavelet name
        wavelet_name = wavelet

        #load kind of data
        if kind == "train" or kind == "full":
            load_train_full_cwt(wavelet_name,scales,kind)
        elif kind == "test":
            load_test_cwt (wavelet_name,scales,kind)

    elif wavelet == "cgau" or wavelet == "gaus":
        @interact(P = IntSlider(min=1, max=8, step=1, value=8))
        def wavelet (P):
            #load wavelet name
            if wavelet == "cgau":
                wavelet_name = "cgau{}".format(P)
            else: 
                wavelet_name = "gaus{}".format(P)
            #load kind of data
            if kind == "train" or kind == "full":
                load_train_full_cwt(wavelet_name,scales,kind)
            elif kind == "test":
                load_test_cwt (wavelet_name,scales,kind)          

interactive(children=(Dropdown(description='kind', options=('train', 'full', 'test'), value='train'), Dropdown…

In [3]:
@interact(B = FloatSlider(min=0.1,max=5,step=0.1,value=1.5), C = FloatSlider(min=0.1,max=5,step=0.1,value=1),P = IntSlider(min=1, max=8, step=1, value=8))
def mother_wavelet(B,C,P):

    #visualize:
    fig,((ax1,ax2),(ax3,ax4),(ax5,_)) = plt.subplots(nrows = 3, ncols = 2, figsize = ( 8,15))
    fig.suptitle("Mother Wavelet Function")

    #axes 1:
    wavelet_name = "cmor{}-{}".format(B,C)
    wavelet = pywt.ContinuousWavelet(wavelet_name)
    fc = pywt.central_frequency(wavelet_name)
    wavelet_value, t = wavelet.wavefun()
    ax1.plot (t,wavelet_value.real)
    ax1.plot (t,wavelet_value.imag)
    ax1.legend(["real","imag"],loc = "upper left")
    ax1.set_title("{}, fc {}".format(wavelet_name,fc))
    ax1.set_xlabel ("Time")
    ax1.set_ylabel ("Value")

    #axes 2:
    wavelet_name = "morl"
    wavelet = pywt.ContinuousWavelet(wavelet_name)
    fc = pywt.central_frequency(wavelet_name)
    wavelet_value, t = wavelet.wavefun()
    ax2.plot (t,wavelet_value)
    ax2.legend(["real"],loc = "upper left")
    ax2.set_title("{}, fc {}".format(wavelet_name,fc))
    ax2.set_xlabel ("Time")
    ax2.set_ylabel ("Value")

    #axes 3:
    wavelet_name = "cgau{}".format(P)
    wavelet = pywt.ContinuousWavelet(wavelet_name)
    fc = pywt.central_frequency(wavelet_name)
    wavelet_value, t = wavelet.wavefun()
    ax3.plot (t,wavelet_value.real)
    ax3.plot (t,wavelet_value.imag)
    ax3.legend(["real","imag"],loc = "upper left")
    ax3.set_title("{}, fc {}".format(wavelet_name,fc))
    ax3.set_xlabel ("Time")
    ax3.set_ylabel ("Value")

    #axes 4:
    wavelet_name = "gaus{}".format(P)
    wavelet = pywt.ContinuousWavelet(wavelet_name)
    fc = pywt.central_frequency(wavelet_name)
    wavelet_value, t = wavelet.wavefun()
    ax4.plot (t,wavelet_value.real)
    ax4.legend(["real"],loc = "upper left")
    ax4.set_title("{}, fc {}".format(wavelet_name,fc))
    ax4.set_xlabel ("Time")
    ax4.set_ylabel ("Value")

    #axes 5:
    wavelet_name = "mexh"
    wavelet = pywt.ContinuousWavelet(wavelet_name)
    fc = pywt.central_frequency(wavelet_name)
    wavelet_value, t = wavelet.wavefun()
    ax5.plot (t,wavelet_value.real)
    ax5.legend(["real"],loc = "upper left")
    ax5.set_title("{}, fc {}".format(wavelet_name,fc))
    ax5.set_xlabel ("Time")
    ax5.set_ylabel ("Value")


interactive(children=(FloatSlider(value=1.5, description='B', max=5.0, min=0.1), FloatSlider(value=1.0, descri…